Install qlatent is required

In [1]:
!pip install qlatent

In [2]:
import importlib
from tqdm.auto import tqdm
from qlatent.qmnli.qmnli import *
from qlatent.qmnli.qmnli import _QMNLI, QMNLI
# importlib.reload(_QMNLI)

import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0


In [3]:
# softmax_files = [False, True]
# softmax_files = [True]

def split_question(Q, index, scales, softmax, filters):
  result = []
  for s in scales:
    q = QCACHE(Q(index=index, scale=s))
    for sf in softmax:
      for f in filters:
        if sf:            
            qsf = QSOFTMAX(q,dim=[index[0], s])
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print((index, s),sf,f)
            result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=s)
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(s,sf,f)
#             result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=index[0])
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(index[0],sf,f)
#             result.append(qsf_f)
        else:
            qsf = QPASS(q,descupdate={'softmax':''})
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print(s,sf,f)
            result.append(qsf_f)
  return result

def dict_pos_neg(pos, neg, w):
  return dict(dict_same_weight(1.0*w/len(pos),pos), **dict_same_weight(-1.0*w/len(neg),neg))

def print_permutations(q):
#     for q in Q1s:
    W = q._pdf['W']
    print(q._descriptor)
    for i, (kmap, w) in enumerate(zip(q._keywords_map, W)):
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
#         sexisem_score = sexisem_classifier(context.strip('.') + ' ' +answer)
        print(f'{i}.',context ,'->', answer, w)
#     break


frequency_weights:SCALE = {
    'never':-4,
    'very rarely':-3,
    'seldom':-2,
    'rarely':-2,
    'frequently':2,
    'often':2,
    'very frequently':3,
    'always':4,    
}
    
    
binary_frequency_weights = {k: 1 if v > 0 else -1 for k, v in frequency_weights.items()}
rbinary_frequency_weights = {k: -1 if v > 0 else 1 for k, v in frequency_weights.items()}
frequency_pos = [k for k, v in frequency_weights.items() if v > 0]
frequency_neg = [k for k, v in frequency_weights.items() if v < 0]

In [4]:
# p = 'valhalla/distilbart-mnli-12-6'
p = "typeform/distilbert-base-uncased-mnli"
mnli = pipeline("zero-shot-classification",device=device, model=p)
mnli.model_identifier = p


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


# ASI

In [5]:
from qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli import asi_bg_qmnli, asi_bi_qmnli, asi_bp_qmnli, asi_h_qmnli

In [6]:
for Q in asi_h_qmnli:
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
2
Query time: 0.09704875946044922
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.05007375574981173
Internal consistency (silhouette, correlation) for unfiltered: 0.549934446811676
Internal consistency (Calinski&Harabasz)  for unfiltered: 6.8382744789123535
Internal consistency (Davies&Bouldin) for unfiltered: 0.6230211776743314


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0002,0.0910,0.1302,0.0926,0.1599,0.1425,0.1864,0.1971
prejudiced,0.0022,0.0922,0.1218,0.0589,0.1699,0.1659,0.1887,0.2004
sexist,0.0568,0.0955,0.1150,0.1091,0.1458,0.1369,0.1736,0.1673
legitimate,0.0076,0.2342,0.1900,0.2329,0.1174,0.1566,0.0516,0.0098
reasonable,0.3000,0.1558,0.1160,0.1525,0.0701,0.0814,0.0554,0.0688
rightful,0.3088,0.1425,0.1145,0.1570,0.0842,0.0868,0.0561,0.0501


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]
['sexist', 'prejudiced', 'improper'] Alpha: 0.9626037845193076
['legitimate', 'rightful', 'reasonable'] Alpha: 0.49545750730899385
Global alpha: 0.8786698602070632
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
4
Query time: 0.012990236282348633
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.1053814844834859
Internal consistency (silhouette, correlation) for unfiltered: 0.8295053839683533
Internal consistency (Calinski&Harabasz)  for unfiltered: 80.00180053710938
Internal consistency (Davies&Bouldin) for unfiltered: 0.19556063352698355


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


...

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
21
Query time: 0.012474775314331055
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.09992761179170075
Internal consistency (silhouette, correlation) for unfiltered: 0.7969953417778015
Internal consistency (Calinski&Harabasz)  for unfiltered: 62.399635314941406
Internal consistency (Davies&Bouldin) for unfiltered: 0.24026840150779696


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0001,0.0689,0.1099,0.1008,0.1846,0.1580,0.1824,0.1955
prejudiced,0.0000,0.0002,0.0430,0.0001,0.2333,0.2292,0.2404,0.2539
sexist,0.0001,0.0006,0.0367,0.0003,0.2426,0.2350,0.2471,0.2376
legitimate,0.2596,0.2763,0.2125,0.2502,0.0002,0.0010,0.0002,0.0001
reasonable,0.3272,0.2278,0.2003,0.2445,0.0000,0.0001,0.0000,0.0000
rightful,0.2468,0.2359,0.1818,0.2054,0.0314,0.0768,0.0215,0.0003


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]
['sexist', 'prejudiced', 'improper'] Alpha: 0.948207691094491
['legitimate', 'rightful', 'reasonable'] Alpha: 0.9820561608183791
Global alpha: 0.9854836264803952


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


# BIG5

In [7]:
from qpsychometric.personality_traits.big5.big5_qmnli import big5_qmnli

In [8]:
for Q in tqdm(big5_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/14 [00:00<?, ?it/s]

(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
1
Query time: 0.012470245361328125
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 0.08601476757596375
Internal consistency (silhouette, correlation) for unfiltered: 0.8695679306983948
Internal consistency (Calinski&Harabasz)  for unfiltered: 127.6586685180664


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


Internal consistency (Davies&Bouldin) for unfiltered: 0.14059553356450225


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
avoid,0.1654,0.1679,0.1656,0.1678,0.0868,0.0829,0.0837,0.0799
dislike,0.2235,0.2145,0.2197,0.2272,0.0314,0.0314,0.0267,0.0257
reject,0.1942,0.1971,0.1955,0.1977,0.0636,0.0403,0.0514,0.0603
am open to,0.0005,0.0130,0.0012,0.0011,0.2324,0.2592,0.2450,0.2477
enjoy,0.0144,0.0015,0.0107,0.0026,0.2355,0.2443,0.2596,0.2314
like,0.0090,0.0110,0.0154,0.0029,0.2339,0.2458,0.2312,0.2508


...

/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


['calmed', 'collected', 'composed'] Alpha: 0.8885307906503078
['stressed', 'worry', 'concern'] Alpha: 0.8376415384833364
Global alpha: 0.9323078396347596
(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
14
Query time: 0.01061391830444336
Mean score unfiltered [-2.0..2.0]: -0.07669776688562706
Internal consistency (silhouette, correlation) for unfiltered: 0.5577860474586487
Internal consistency (Calinski&Harabasz)  for unfiltered: 12.89246940612793
Internal consistency (Davies&Bouldin) for unfiltered: 0.45624449986707294


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
calmed,0.0004,0.0021,0.0003,0.0004,0.2477,0.2604,0.2352,0.2535
relaxed,0.0350,0.0936,0.0615,0.0894,0.1826,0.1841,0.1707,0.1832
agitated,0.1884,0.1755,0.1884,0.1675,0.0752,0.0697,0.0741,0.0612
prone to mood swings,0.1680,0.1199,0.1065,0.1481,0.1019,0.0931,0.1260,0.1365
upset,0.2088,0.1958,0.2232,0.1913,0.0466,0.0473,0.0529,0.0341


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['calmed', 'relaxed'] Alpha: 0.8449170005105837
['upset', 'prone to mood swings', 'agitated'] Alpha: 0.7999160268965742
Global alpha: 0.5726170448615263


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
100%|██████████| 14/14 [00:00<00:00, 14.53it/s]


# SD3

In [9]:
from qpsychometric.personality_traits.short_dark_triad.sd3_qmnli import sd3_qmnli

In [10]:
for Q in tqdm(sd3_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/27 [00:00<?, ?it/s]

(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
1
Query time: 0.010064363479614258
Mean score unfiltered [-2.0..2.0]: -0.09148252894692632
Internal consistency (silhouette, correlation) for unfiltered: 0.18062667548656464
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.3965208530426025
Internal consistency (Davies&Bouldin) for unfiltered: 0.8979169101255574


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
good,0.0010,0.1728,0.1177,0.0991,0.1336,0.1476,0.1175,0.2108
kind,0.0008,0.0248,0.0072,0.0017,0.2328,0.2179,0.2357,0.2791
hostil,0.0226,0.1571,0.2119,0.2291,0.1083,0.0870,0.1096,0.0744
mean,0.3953,0.1255,0.1320,0.1351,0.0616,0.0793,0.0711,0.0001


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['good', 'kind'] Alpha: 0.6099153975295761
['mean', 'hostil'] Alpha: -0.6411077622043182
Global alpha: 0.7122089073616211


...

/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


Internal consistency (Davies&Bouldin) for unfiltered: 0.8638375829182348


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
false,0.3115,0.1229,0.1210,0.1189,0.0889,0.1047,0.1025,0.0297
incorrect,0.0310,0.1624,0.1637,0.1538,0.1342,0.1636,0.1475,0.0439
correct,0.0728,0.1118,0.1125,0.1280,0.1234,0.1120,0.1136,0.2259
true,0.0160,0.1120,0.1128,0.1061,0.1666,0.1347,0.1490,0.2028


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
100%|██████████| 27/27 [00:01<00:00, 14.95it/s]

index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['incorrect', 'false'] Alpha: -1.082834339294934
['correct', 'true'] Alpha: 0.8712557708720206
Global alpha: 0.668324188020431


# SOC

In [11]:
from qpsychometric.mental_health.sense_of_coherence.soc_qmnli import soc_qmnli

In [12]:
for Q in tqdm(soc_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/12 [00:00<?, ?it/s]

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
4
Query time: 0.012916088104248047
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 0.09386753136861141
Internal consistency (silhouette, correlation) for unfiltered: 0.8371579051017761
Internal consistency (Calinski&Harabasz)  for unfiltered: 83.18085479736328
Internal consistency (Davies&Bouldin) for unfiltered: 0.1945987855762453


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
out of control,0.3248,0.2430,0.2190,0.2095,0.0001,0.0035,0.0001,0.0000
uncontrollable,0.2204,0.1556,0.2587,0.2654,0.0012,0.0976,0.0011,0.0001
unmanageable,0.3244,0.2131,0.2042,0.2318,0.0003,0.0258,0.0003,0.0001
collected,0.0000,0.0473,0.0425,0.0347,0.2329,0.1873,0.2494,0.2060
controlled,0.0000,0.0741,0.0612,0.0613,0.2015,0.2008,0.1839,0.2172
contained,0.0000,0.0711,0.0632,0.0553,0.2041,0.1835,0.2018,0.2211


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]
['out of control', 'uncontrollable', 'unmanageable'] Alpha: 0.9708197953991966
['contained', 'collected', 'controlled'] Alpha: 0.9875867273871344
Global alpha: 0.9850872743577188
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
5


...

Internal consistency (silhouette, correlation) for unfiltered: 0.7453703284263611
Internal consistency (Calinski&Harabasz)  for unfiltered: 37.92216110229492
Internal consistency (Davies&Bouldin) for unfiltered: 0.28094844944290287


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
out of control,0.4112,0.1888,0.1888,0.1801,0.0001,0.0308,0.0001,0.0000
uncontrollable,0.2013,0.1595,0.2510,0.2572,0.0059,0.1201,0.0050,0.0001
unmanageable,0.3016,0.1774,0.2034,0.2293,0.0017,0.0837,0.0028,0.0001
collected,0.0000,0.0735,0.0564,0.0525,0.2318,0.1614,0.2182,0.2062
controlled,0.0054,0.1223,0.1008,0.0981,0.1664,0.1634,0.1903,0.1532
contained,0.0001,0.0909,0.0750,0.0716,0.1893,0.1545,0.1858,0.2329


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
100%|██████████| 12/12 [00:00<00:00, 14.22it/s]

index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]
['out of control', 'uncontrollable', 'unmanageable'] Alpha: 0.9471823753919999
['contained', 'collected', 'controlled'] Alpha: 0.958310687719593
Global alpha: 0.9669734321318936


# PHQ

In [13]:
from qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli import phq_qmnli

In [14]:
for Q in tqdm(phq_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/9 [00:00<?, ?it/s]

(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
1
Query time: 0.013159036636352539
Mean score unfiltered [-2.0..2.0]: -0.11203825167910964
Internal consistency (silhouette, correlation) for unfiltered: 0.8797760605812073
Internal consistency (Calinski&Harabasz)  for unfiltered: 120.51805114746094
Internal consistency (Davies&Bouldin) for unfiltered: 0.11341018731029834


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
satisfaction,0.0000,0.0083,0.0028,0.0002,0.2394,0.2368,0.2488,0.2638
pleasure,0.0000,0.0158,0.0058,0.0005,0.2565,0.2458,0.2460,0.2296
displeasure,0.3320,0.1983,0.2359,0.2303,0.0001,0.0031,0.0002,0.0000
dissatisfaction,0.2619,0.2247,0.2542,0.2552,0.0002,0.0035,0.0003,0.0000
no pleasure,0.2480,0.2485,0.2390,0.2509,0.0002,0.0131,0.0002,0.0000
little interest,0.1644,0.2901,0.2655,0.2733,0.0001,0.0063,0.0002,0.0001


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['pleasure', 'satisfaction'] Alpha: 0.996542835693706
['little interest', 'no pleasure', 'displeasure', 'dissatisfaction'] Alpha: 0.9860924179522246
Global alpha: 0.28824850298661514


...

index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['normally', 'naturally'] Alpha: 0.9971519826604083
['fidgetly', 'slowly'] Alpha: 0.5216317143054201
Global alpha: 0.9314835375369208
(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
9
Query time: 0.012823820114135742
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.10564941330231804
Internal consistency (silhouette, correlation) for unfiltered: 0.9174527525901794
Internal consistency (Calinski&Harabasz)  for unfiltered: 406.136962890625
Internal consistency (Davies&Bouldin) for unfiltered: 0.09424678446779024


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
happy,0.0000,0.0162,0.0221,0.0006,0.2419,0.2379,0.2446,0.2367
hopeful,0.0000,0.0378,0.0401,0.0109,0.2351,0.2349,0.2195,0.2217
positive,0.0000,0.0277,0.0245,0.0014,0.2342,0.2154,0.2358,0.2610
deadly,0.2723,0.2349,0.2330,0.2521,0.0002,0.0073,0.0002,0.0000
self destructive,0.2486,0.2502,0.2015,0.2374,0.0137,0.0236,0.0249,0.0000
suicidal,0.2549,0.2028,0.2519,0.2777,0.0004,0.0109,0.0012,0.0000


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['happy', 'hopeful', 'positive'] Alpha: 0.9966870056390094
['suicidal', 'self destructive', 'deadly'] Alpha: 0.9936937370420487
Global alpha: 0.99776489849807


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
100%|██████████| 9/9 [00:00<00:00, 14.40it/s]


# GAD

In [15]:
from qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli import gad_qmnli

In [16]:
for Q in tqdm(gad_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/7 [00:00<?, ?it/s]

(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
1
Query time: 0.01262521743774414
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.06719284413177067
Internal consistency (silhouette, correlation) for unfiltered: 0.5541771650314331
Internal consistency (Calinski&Harabasz)  for unfiltered: 6.506024360656738
Internal consistency (Davies&Bouldin) for unfiltered: 0.5669283237362106


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
calm,0.0004,0.1106,0.0877,0.0916,0.1740,0.1416,0.1780,0.2161
peaceful,0.0007,0.1138,0.0904,0.0958,0.1730,0.1478,0.1658,0.2126
relaxed,0.0002,0.0853,0.0832,0.0898,0.1846,0.1523,0.1918,0.2128
anxious,0.2973,0.1558,0.1356,0.1208,0.0954,0.0943,0.0979,0.0031
on edge,0.0138,0.1567,0.2758,0.3308,0.0358,0.1674,0.0192,0.0004
nervous,0.4720,0.1530,0.1708,0.1350,0.0090,0.0503,0.0098,0.0001


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


...

['calm', 'tranquil', 'peaceful', 'relaxed'] Alpha: 0.9756808083367432
['annoyed', 'irritated', 'frustrated', 'bothered'] Alpha: 0.8804849329239384
Global alpha: 0.9551226296349598
(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
7
Query time: 0.011438846588134766
Mean score unfiltered [-2.0..2.0]: -0.10238713551486096
Internal consistency (silhouette, correlation) for unfiltered: 0.678034782409668
Internal consistency (Calinski&Harabasz)  for unfiltered: 20.27925682067871
Internal consistency (Davies&Bouldin) for unfiltered: 0.37029180703735937


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
calm,0.0038,0.1006,0.0823,0.0845,0.1837,0.1698,0.1865,0.1888
relaxed,0.0017,0.1128,0.0932,0.0878,0.1983,0.1966,0.1883,0.1212
tranquil,0.0010,0.0696,0.0643,0.0350,0.1907,0.1850,0.1963,0.2582
afraid,0.4364,0.1677,0.1880,0.2059,0.0002,0.0014,0.0003,0.0000
scared,0.1908,0.2073,0.2414,0.2639,0.0208,0.0562,0.0195,0.0002


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
100%|██████████| 7/7 [00:00<00:00, 14.39it/s]

index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['calm', 'tranquil', 'relaxed'] Alpha: 0.9417120073257529
['afraid', 'scared'] Alpha: 0.8377166043402882
Global alpha: 0.6071374036334884


# Compassion Scale

In [17]:
from qpsychometric.personality_traits.compassion_scale.compassion_scale_qmnli import compassion_scale_qmnli

In [18]:
for Q in tqdm(compassion_scale_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/24 [00:00<?, ?it/s]

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
1
Query time: 0.018175601959228516
Mean score unfiltered [-1.0..1.0]: -0.04708116650856228
Internal consistency (silhouette, correlation) for unfiltered: 0.6196066737174988
Internal consistency (Calinski&Harabasz)  for unfiltered: 7.141218662261963
Internal consistency (Davies&Bouldin) for unfiltered: 0.6629391238658997


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
a minor,0.0017,0.0015,0.0017,0.0024,0.0453,0.0723,0.1890,0.6860
an irrelevant,0.0023,0.0032,0.0026,0.0020,0.0053,0.0113,0.4058,0.5675
an insignificant,0.0160,0.0218,0.0196,0.0146,0.0366,0.0668,0.1297,0.6948
an unimportant,0.0020,0.0050,0.0020,0.0018,0.0084,0.0085,0.9245,0.0477
a significant,0.1196,0.1339,0.1238,0.1246,0.1276,0.1150,0.1344,0.1212
a crucial,0.1428,0.1257,0.1310,0.1330,0.1238,0.1237,0.1104,0.1095
an important,0.1237,0.1272,0.1308,0.1279,0.1277,0.1279,0.1210,0.1136
a critical,0.1188,0.1198,0.1215,0.1207,0.1323,0.1376,0.1256,0.1236
an essential,0.1444,0.1295,0.1351,0.1366,0.1204,0.1330,0.1050,0.0959


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]
['an insignificant', 'an unimportant', 'an irrelevant', 'a minor'] Alpha: 0.8089224009376317
['an important', 'a crucial', 'an essential', 'a significant', 'a critical'] Alpha: 0.49417357180008625
Global alpha: 0.7199731435043788


...

Query time: 0.01777362823486328
Mean score unfiltered [-1.0..1.0]: 0.023985262684355906
Internal consistency (silhouette, correlation) for unfiltered: 0.3456767797470093
Internal consistency (Calinski&Harabasz)  for unfiltered: 7.972084045410156
Internal consistency (Davies&Bouldin) for unfiltered: 0.8582384696065152


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
a minor,0.1316,0.1305,0.1305,0.1263,0.1177,0.1340,0.1341,0.0953
an irrelevant,0.2186,0.2469,0.2378,0.2157,0.0381,0.0090,0.0318,0.0021
an insignificant,0.1213,0.1286,0.1274,0.1169,0.1363,0.1249,0.1237,0.1210
an unimportant,0.2170,0.2280,0.2236,0.2267,0.0276,0.0375,0.0378,0.0018
a significant,0.1071,0.1173,0.1148,0.1115,0.1306,0.1433,0.1311,0.1444
a crucial,0.1289,0.1097,0.1128,0.1207,0.1375,0.1338,0.1360,0.1205
an important,0.0853,0.0836,0.0908,0.0946,0.1557,0.1574,0.1511,0.1816
a critical,0.1049,0.1060,0.1055,0.1100,0.1416,0.1363,0.1433,0.1524
an essential,0.0862,0.0635,0.0663,0.0826,0.1637,0.1696,0.1640,0.2041


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]
['an insignificant', 'an unimportant', 'an irrelevant', 'a minor'] Alpha: 0.6885614028700143
['an important', 'a crucial', 'an essential', 'a significant', 'a critical'] Alpha: 0.8801835784300307
Global alpha: 0.8472060052361663


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
100%|██████████| 24/24 [00:01<00:00, 12.79it/s]


# Load all questions

In [19]:
import sys, importlib
import qpsychometric
importlib.reload(sys.modules['qpsychometric'])
from qpsychometric.utils import load_questions

<module 'qpsychometric' from '/sise/home/koremmi/qpsychometric/qpsychometric/__init__.py'>

In [20]:
all_qmnli = load_questions()

load: asi_qmnli
load: big5_qmnli
load: sd3_qmnli
load: compassion_scale_qmnli
load: soc_qmnli
load: phq_qmnli
load: gad_qmnli


In [21]:
for Q in tqdm(all_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/115 [00:00<?, ?it/s]

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
1
Query time: 0.016880035400390625
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.08677949091755484
Internal consistency (silhouette, correlation) for unfiltered: 0.6724789142608643
Internal consistency (Calinski&Harabasz)  for unfiltered: 21.75181770324707
Internal consistency (Davies&Bouldin) for unfiltered: 0.38562455967666526


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0006,0.1114,0.0906,0.0796,0.1756,0.1518,0.1700,0.2203
prejudiced,0.0006,0.0596,0.0566,0.0037,0.2441,0.2131,0.2637,0.1586
sexist,0.0020,0.0946,0.0707,0.0473,0.2007,0.1641,0.2091,0.2115
legitimate,0.1783,0.1891,0.2886,0.3398,0.0005,0.0025,0.0003,0.0009
reasonable,0.4393,0.1335,0.1732,0.2112,0.0004,0.0407,0.0002,0.0016
rightful,0.2605,0.2061,0.1728,0.2066,0.0237,0.0991,0.0004,0.0307


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


...

index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['calm', 'tranquil', 'peaceful', 'relaxed'] Alpha: 0.9756808083367432
['annoyed', 'irritated', 'frustrated', 'bothered'] Alpha: 0.8804849329239384
Global alpha: 0.9551226296349598
(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
7
Query time: 0.011700630187988281
Mean score unfiltered [-2.0..2.0]: -0.10238713551486096
Internal consistency (silhouette, correlation) for unfiltered: 0.678034782409668
Internal consistency (Calinski&Harabasz)  for unfiltered: 20.27925682067871
Internal consistency (Davies&Bouldin) for unfiltered: 0.37029180703735937


index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
calm,0.0038,0.1006,0.0823,0.0845,0.1837,0.1698,0.1865,0.1888
relaxed,0.0017,0.1128,0.0932,0.0878,0.1983,0.1966,0.1883,0.1212
tranquil,0.0010,0.0696,0.0643,0.0350,0.1907,0.1850,0.1963,0.2582
afraid,0.4364,0.1677,0.1880,0.2059,0.0002,0.0014,0.0003,0.0000
scared,0.1908,0.2073,0.2414,0.2639,0.0208,0.0562,0.0195,0.0002


/home/koremmi/.conda/envs/michael_projects/lib/python3.12/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f31e817fe20> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
100%|██████████| 115/115 [00:08<00:00, 14.04it/s]

index = ['emotion']
{'intensifier', 'emotion'} {'intensifier'} {'emotion'}
[]
['calm', 'tranquil', 'relaxed'] Alpha: 0.9417120073257529
['afraid', 'scared'] Alpha: 0.8377166043402882
Global alpha: 0.6071374036334884
